In [ ]:
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call,ModelResponse,ModelRequest

@wrap_model_call
def log_model(request:ModelRequest,handler) -> ModelResponse:
    print("模型调用前".center(50,"="))
    print("输入内容：",[m.content for m in request.messages])

    response = handler(request)

    print("模型调用后".center(50,"="))
    print("模型响应内容：",response.result)

    return response


model = ChatOllama(model="qwen3:1.7b")
agent = create_agent(model=model,middleware=[log_model])

agent.invoke({
    "messages": [HumanMessage("介绍一下你自己")]
})

In [ ]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

@tool
def send_email(recipient:str,subject:str,body:str) -> str:
    """发送电子邮件，此操作需要人工审核"""
    print("发送邮件".center(50,"="))
    print(f"模拟发送邮件给{recipient},主题：{subject}")
    print("邮件内容：",body)
    return f"邮件已经发送给{recipient}"

def human_review(tool_name, input_args):
    print(f"\n⚠️ 工具调用请求: {tool_name}")
    print(f"参数: {input_args}")
    decision = input("批准发送邮件吗？(approve/edit/reject): ")

    if decision == "edit":
        new_body = input("请输入修改后的邮件内容: ")
        input_args["body"] = new_body
        return {"decision": "approve", "input_args": input_args}
    elif decision == "reject":
        return {"decision": "reject"}
    else:
        return {"decision": "approve"}




human_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "send_email": {
            "allowed_decisions": ["approve", "edit", "reject"],
            "callback": human_review,
        }
    }
)

model = ChatOllama(model="qwen3:1.7b",format="json")
agent = create_agent(model=model,tools=[send_email],middleware=[human_middleware],checkpointer=InMemorySaver())

response =  agent.invoke({
    "messages": [HumanMessage("请帮我发一封邮件给 Alice，说：'会议改到明天下午三点'。注意你必须使用工具 send_email 来发送邮件")]
},config={
    "thread_id":"thread-1"
})

print("调用工具：",response)

In [30]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_ollama import ChatOllama
from langchain.agents.middleware import HumanInTheLoopMiddleware

@tool
def search_web(query: str) -> str:
    """在网络上搜索信息。"""
    return f"关于 '{query}' 的搜索结果：相关文章、新闻和信息"

@tool
def get_weather(location: str) -> str:
    """获取指定位置的天气信息。"""
    print("获取天气信息的工具".center(50,"="))
    return f"{location}的天气：晴朗，25°C"


human_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "get_weather": {
            "allowed_decisions": ["approve", "edit", "reject"],
        }
    }
)

model = ChatOllama(model="qwen3:1.7b")
# 创建基础 Agent
agent = create_agent(
    model=model,
    tools=[search_web, get_weather],
    middleware=[human_middleware],
)

response = agent.invoke({
    "messages": [HumanMessage("北京的天气怎么样")]
})

print(response['messages'][1])

content='' additional_kwargs={} response_metadata={'model': 'qwen3:1.7b', 'created_at': '2025-10-22T11:08:23.4638227Z', 'done': True, 'done_reason': 'stop', 'total_duration': 702470800, 'load_duration': 45301800, 'prompt_eval_count': 183, 'prompt_eval_duration': 130406500, 'eval_count': 97, 'eval_duration': 499115300, 'model_name': 'qwen3:1.7b', 'model_provider': 'ollama'} id='lc_run--26f04450-b49a-47aa-b758-eac8eec4dc58-0' tool_calls=[{'name': 'get_weather', 'args': {'location': '北京'}, 'id': 'f3af386b-9d65-4518-a2ec-e2203eabac75', 'type': 'tool_call'}] usage_metadata={'input_tokens': 183, 'output_tokens': 97, 'total_tokens': 280}


In [ ]:
response['']